In [1]:
import pandas as pd
import numpy as np
from random import seed

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

from wittgenstein.irep import IREP
from wittgenstein.ripper import RIPPER
from wittgenstein.interpret import interpret_model, score_fidelity

In [2]:
inpath = ''
df = pd.read_csv(inpath+"wine.csv")
df.head()

,Class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [3]:
# Train-test-split

X, y = df.drop('Class', axis=1), df['Class']
y = y.map(lambda x: 1 if x==1 else 0)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42
)

In [4]:
# Create an interpreter

interpreter = RIPPER(random_state=42)

In [5]:
# Interpret an SVM

svc = SVC(kernel='rbf', random_state=42)
svc.fit(X_train, y_train);

interpret_model(model=svc, X=X_train, interpreter=interpreter)
interpreter.out_model()

[[Proline=>1227.0] V
[Proline=1048.0-1227.0] V
[Proline=880.0-1048.0]]


In [6]:
score_fidelity(
    X_test,
    interpreter,
    model=svc,
    score_function=[precision_score, recall_score, f1_score])

[1.0, 1.0, 1.0]

In [7]:
# Interpret a Random Forest

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

ruleset, _ = interpret_model(model=rf, X=X_train, interpreter=interpreter)
interpreter.out_model()

[[Proline=>1227.0] V
[Proline=1048.0-1227.0] V
[Proline=880.0-1048.0 ^ Ash=2.0-2.19] V
[Proline=880.0-1048.0 ^ Malicacid=1.76-1.9] V
[Colorintensity=5.75-6.78]]


In [8]:
score_fidelity(
    X_test,
    interpreter,
    model=rf,
    score_function=[precision_score, recall_score, f1_score])

[1.0, 0.7333333333333333, 0.846153846153846]

In [9]:
# Interpret a Tensorflow/Keras model

seed(1)
wine_df = pd.read_csv(inpath + "wine.csv")
X_wine, y_wine = wine_df.drop("Class", axis=1), wine_df["Class"]
y_wine = y_wine.map(lambda x: True if x==1 else False)
X_train_wine, X_test_wine, y_train_wine, y_test_wine = train_test_split(X, y, random_state=42)

mlp = Sequential()
mlp.add(Dense(60, input_dim=13, activation='relu'))
mlp.add(Dense(30, activation='relu'))
mlp.add(Dense(1, activation='sigmoid'))
mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
mlp.fit(
    X_train_wine,
    y_train_wine,
    batch_size=1,
    epochs=10,
)

ruleset, _ = interpret_model(model=mlp, X=X_train, interpreter=interpreter)
interpreter.out_model()

Epoch 1/10
133/133 [==============================] - 0s 1ms/step - loss: 3.4373 - accuracy: 0.6692
Epoch 2/10
133/133 [==============================] - 0s 1ms/step - loss: 2.6298 - accuracy: 0.6466
Epoch 3/10
133/133 [==============================] - 0s 2ms/step - loss: 0.7250 - accuracy: 0.8271
Epoch 4/10
133/133 [==============================] - 0s 1ms/step - loss: 0.8538 - accuracy: 0.8271
Epoch 5/10
133/133 [==============================] - 0s 1ms/step - loss: 0.8990 - accuracy: 0.8571
Epoch 6/10
133/133 [==============================] - 0s 1ms/step - loss: 0.8094 - accuracy: 0.8797
Epoch 7/10
133/133 [==============================] - 0s 1ms/step - loss: 1.0302 - accuracy: 0.8571
Epoch 8/10
133/133 [==============================] - 0s 1ms/step - loss: 0.8946 - accuracy: 0.9023
Epoch 9/10
133/133 [==============================] - 0s 1ms/step - loss: 0.5973 - accuracy: 0.8722
Epoch 10/10
133/133 [==============================] - 0s 2ms/step - loss: 1.3158 - accuracy: 0.8120

In [10]:
score_fidelity(
    X_test,
    interpreter,
    model=mlp,
    score_function=[precision_score, recall_score, f1_score])

[0.85, 0.9444444444444444, 0.8947368421052632]